In [2]:
import os
import json
import random
import logging
# import pandas as pd
import spacy
from tqdm import tqdm
from spacy.tokens import DocBin

VISUALIZATION_STATUS = True

In [3]:
def convert_spacy_format(data, output_path="/home/ehz/nlp-process/data/unkonwn_ehza.spacy"):
    nlp = spacy.blank("bn") # load a new spacy model
    db = DocBin() # create a DocBin object
    c = 0
    for text, annot in tqdm(data): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            
            span = doc.char_span(start, end, label=label, alignment_mode="strict")
            # print(start, end, label, span)
            if span is None:
                s = doc.text
                sub_E = s[end:]
                sub_S = s[:start]
                end = end+ (0 if len(sub_E.split(" ", 1)[0]) <= 0 else len(sub_E.split(" ", 1)[0]))
                start = start - (0 if len(sub_S.rsplit(" ", 1)[-1]) <= 0 else len(sub_S.rsplit(" ", 1)[-1]))
                
                span = doc.char_span(start, end, label=label, alignment_mode="strict")
                # print("Text:", text)
                # print("============:", s[start:end], "span:", span, label)
                if span is None:
                    print("++++++++++++++++++++++++++++Skipping entity Start++++++++++++++++++++++++++++")
                    print(start, end, label, span)
                    print(doc.text[start:end],doc.text[start],doc.text[end-1],'kh',sep='|')
                    print("++++++++++++++++++++++++++++Skipping entity End++++++++++++++++++++++++++++++")
                    break
                    c+=1
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        if VISUALIZATION_STATUS:
            spacy.displacy.render(doc, style="ent", jupyter=True)
        db.add(doc)
    db.to_disk(output_path) # save the docbin object

In [2]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.5 MB/s eta 0:00:00m eta 0:00:010:01:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.1.1
    Uninstalling pydantic-2.1.1:
      Successfully uninstalled pydantic-2.1.1


In [4]:
def convert_doccano_to_spacy(doccano_JSON_FilePath):
    # try:
    training_data = []
    lines=[]
    with open(doccano_JSON_FilePath, 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        # print(data)
        text = data['text']
        entities = data['label']
        if len(entities)>0:
            training_data.append((text, {"entities" : entities}))
    return training_data

In [5]:
def save_jsonl(data, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for item in data:
            json.dump(item, file, ensure_ascii=False)
            file.write('\n')

In [6]:
data_file_path = "/home/ehz/nlp-processing/outputfile.jsonl"

output_dir = "/home/ehz/nlp-processing/data"

training_percentage = 95

data = convert_doccano_to_spacy(data_file_path)
random.shuffle(data)


train_lenght = int((len(data)*training_percentage)/100)


training_data = data[:train_lenght]
validation_data = data[train_lenght:]

print("Total Number of data :", len(data))
print("Training Data:",len(training_data))
print("Validation Data:",len(validation_data))

save_jsonl(training_data, os.path.join(output_dir, "train.jsonl"))
save_jsonl(training_data, os.path.join(output_dir,"val.jsonl"))
convert_spacy_format(training_data, output_path=os.path.join(output_dir,"train.spacy"))
convert_spacy_format(validation_data, output_path=os.path.join(output_dir,"val.spacy"))

Total Number of data : 3349
Training Data: 3181
Validation Data: 168


FileNotFoundError: [Errno 2] No such file or directory: 'data/train.jsonl'